# Etude des dépendances en températures entre deux positions géographiques

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Suppression des relevés pour les 29/02 des années bissextiles
path = "Data/ukcp09_gridded-land-obs-daily_timeseries_mean-temperature_000000E_500000N_19600101-20161231.csv"
data = pd.read_csv(path, header=[0,1], index_col=0, parse_dates=True)
dates_to_remove = [datetime.date(datetime.strptime(str(y)+"-02-29", '%Y-%m-%d')) for y in range(1960, 2017, 4)]
data = data.drop(dates_to_remove)
# Lecture de la série des résidus (enregistrée indépendemment)
serie1 = pd.read_csv("BB22500-547500.csv", header=None)[1]
serie1.index = data.index
serie2 = pd.read_csv("BB17500-502500.csv", header=None)[1]
serie2.index = data.index
serie2.head()

In [ ]:
# On ne garde qu'une valeur sur 5
serie1 = serie1.values[list(range(0,len(serie1),5))]
serie2 = serie2.values[list(range(0,len(serie2),5))]

# Corrélation linéaire

In [ ]:
correlation = (np.mean(serie1 * serie2) - np.mean(serie1)*np.mean(serie2))/(np.std(serie1)*np.std(serie2))
print("Correlation : "+str(correlation))

# Corrélation des rangs

In [ ]:
# Coefficient de Spearman
n = len(serie1)
argsort1 = np.argsort(serie1)
ranks1 = {}
for k in range(len(argsort1)):
    ranks1[serie1[argsort1[k]]] = k
    
argsort2 = np.argsort(serie2)
ranks2 = {}
for k in range(len(argsort2)):
    ranks2[serie2[argsort2[k]]] = k

In [ ]:
tmp1 = np.array([ranks1[x] for x in serie1])
tmp2 = np.array([ranks2[x] for x in serie2])
rho_s = 12/(n**2-1)*1/n*np.sum((tmp1-(n+1)/2)*(tmp2-(n+1)/2))
print("Coefficient de Spearman : "+str(rho_s))

In [ ]:
# Taux de Kendall
t = 0
for i in range(n):
    for j in range(i,n):
        t += np.sign((serie1[i]-serie1[j])*(serie2[i]-serie2[j]))
t /= n*(n-1)/2
print("Taux de Kendall : "+str(t))

Tous ces indicateurs témoignent d'une corrélation très forte entre les deux séries temporelles

# Copules

## Copule de Gumbel

In [ ]:
# Estimation par le taux de Kendall
theta_g = 1/(1-t)
print("Theta (copule de Gumbel) : "+str(theta_g))

In [ ]:
# Indépendance asymptotique à gauche
lbda_u = 2-2**(1/theta_g)
print("Coefficient de dépendance extrême à droite : " + str(lbda_u))

In [ ]:
# Par maximum de vraisemblance
# Fonctions de répartition empiriques
def F1(x):
    k=0
    n=len(ranks1)
    while((k<n)and(serie1[argsort1[k]]<x)):
        k+=1
    return(k/n)

def F2(x):
    k=0
    n=len(ranks2)
    while((k<n)and(serie2[argsort2[k]]<x)):
        k+=1
    return(k/n)

In [ ]:
U=[(F1(serie1[k]),F2(serie2[k])) for k in range (len(serie1))]

In [ ]:
#Expression copule de Grumbel
def C_gr(theta,v1,v2):
    if (v1*v2!=0):
        a=(-np.log(v1))**theta+(-np.log(v2))**theta
        return(np.exp(-a**(1/theta)))
    else:
        return(0)   

In [ ]:
def phi_g(theta, u):
    return (-np.log(u))**theta

def densite_g(theta, u, v):
    C = C_gr(theta, u, v)
    phi_u = phi_g(theta, u)
    phi_v = phi_g(theta, v)
    phi_u2 = phi_g(theta-1, u)
    phi_v2 = phi_g(theta-1, v)
    return C*(phi_u + phi_v)**(1/theta-2)*(theta-1 + (phi_u + phi_v)**(1/theta))*(phi_u2*phi_v2)/(u*v)

In [ ]:
#Estimation de theta
n=len(serie1)

# gumbel
def vrais_gr(theta):
    res=0
    for k in range(n):
        if U[k][0] != 0 and U[k][1]!=0:
            res+=np.log(densite_g(theta,U[k][0],U[k][1]))
    return(res)

In [ ]:
x = np.linspace(1.1,30,100)
y = []
for theta in x:
    y.append(vrais_gr(theta))
plt.plot(x,y)
plt.show()

In [ ]:
# Theta optimal
thetag2 = x[np.argmax(y)]
print("Theta : " + str(thetag2))
lbda_u2 = 2-2**(1/thetag2)
print("Coefficient de dépendance extrême à droite : " + str(lbda_u2))
print("Tau de Kendall : " + str(1-1/thetag2))

## Copule de Clayton

In [ ]:
# Estimation par le taux de Kendall
theta_c = 2*t/(1-t)
print("Theta (copule de Clayton) : "+str(theta_c))

In [ ]:
# Indépendance asymptotique à droite
lbda_l = 2**(-1/theta_c)
print("Coefficient de dépendance extrême à gauche : " + str(lbda_l))

In [ ]:
# Estimation par maximum de vraisemblance
# Expression copule de Clayton
def C_cl(theta,v1,v2):
    if (v1*v2!=0):
        a=v1**(-theta)+v2**(-theta)-1
        return(a**(-1/theta))
    else:
        return(0)
    
#densité copule clayton :

def densite_cl(theta, u, v):
    return (theta + 1)*(u*v)**(-theta - 1) * (u**(-theta) + v**(-theta)-1)**(-1/theta-2)

In [ ]:
# Log-vraisemblance Clayton
def vrais_cl(theta):
    res=0
    for k in range(n):
        if U[k][0] != 0 and U[k][1] != 0:
            res+=np.log(densite_cl(theta,U[k][0],U[k][1]))
    return(res)

In [ ]:
x = np.linspace(0.1,30,100)
y = []
for theta in x:
    y.append(vrais_cl(theta))
plt.plot(x,y)
plt.show()

In [ ]:
# Theta optimal
thetacl2 = x[np.argmax(y)]
print("Theta : " + str(thetacl2))
lbda_l2 = 2**(-1/thetacl2)
print("Coefficient de dépendance extrême à droite : " + str(lbda_l2))
print("Tau de Kendall : " + str(thetacl2/(thetacl2+2)))